In [1]:
import pandas as pd
import csv
import pickle
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import os
from itertools import product

In [2]:
base_dir = Path(os.getenv("LASSO_OUTPUT_DIR", "output"))

In [3]:
features_path = Path("output") / "features.pkl"
response_path = Path("output") / "response.pkl"
return_path = Path("data") / "return84_20.csv"

with features_path.open("rb") as f:
    X = pickle.load(f)
    
with response_path.open("rb") as f:
    y = pickle.load(f)

if not return_path.exists():
    return_path = Path(r"C:\Users\jonat\Lasso_paper\Empirical\data\return84_20.csv")
    print("Using absolute path:", return_path)

y = np.log(y+1)

Using absolute path: C:\Users\jonat\Lasso_paper\Empirical\data\return84_20.csv


In [4]:
from stage1 import (
    lasso_rolling_window,
    calculate_r_squared
)

In [ ]:
param_grid = {
    'window_sizes': [30, 50],
    'n_lags': [3],
    'lambdas': [0.001]
}

In [5]:
lasso_results = lasso_rolling_window(
            X=X, y=y, 
            window_size=30, 
            n_lags=3,
            lambda_mode="fixed", 
            fixed_lambda=0.001, 
            verbose=True
        )

Creating lagged features with 3 lags...
Running 8407 rolling windows of size 30...
Lambda selection: fixed (α=0.001)


Rolling windows: 100%|██████████| 8407/8407 [00:20<00:00, 403.84it/s]

Rolling LASSO complete!
Average lambda: 0.001000


In [6]:
preds = np.array(lasso_results["predictions"])
y_valid = y[-len(preds):] 
y_vals = y_valid.values 

In [8]:
from stage2 import compute_stage2_r_squared

In [9]:
r2_oos_stage1 = calculate_r_squared(y_vals, preds)
r2_insample_stage1 = np.mean(lasso_results['insample_r_squareds'])
        
        # Stage 2
stage2_input = pd.DataFrame({"vwretd": y_vals, "predictions": preds})
stage2_results = compute_stage2_r_squared(stage2_input, min_train_size=100)